In [1]:
from thesis_initialise import *

In [2]:
import itertools as _itertools

from everest.funcy import generic as _generic
from everest.funcy.base import Base as _Base

from collections.abc import Collection as _Collection

class HierCont(_Collection):
    def __repr__(self):
        return f"{self.__class__.__name__}({super().__repr__()})"
    def _getvals(self):
        return self
    @property
    def shape(self):
        try:
            return self._shape
        except AttributeError:
            vals = self._getvals()
            self._shape = shape = tuple((
                sum(1 for o in self if not isinstance(o, HierCont)),
                tuple(o.shape for o in self if isinstance(o, HierCont)),
                ))
            return shape
class HierDict(HierCont, dict):
    def _getvals(self):
        return tuple(self.values())
class HierList(HierCont, list):
    ...

def hierarchicalise_muddle(mud, /, n = 1_000_000):
    mud = sorted(_itertools.islice(mud, n + 1))
    if len(mud) > n:
        raise ValueError(f"Sequence exceeded max length {n}")
    rowLen = len(mud[0])
    assert rowLen > 0
    if rowLen == 1:
        out = HierList(row[0] for row in mud)
    else:
        out = HierDict()
        for row in mud:
            d = out
            *row, lastkey, lastval = row
            for o in row:
                d = d.setdefault(o, HierDict())
            ls = d.setdefault(lastkey, HierList())
            ls.append(lastval)
    return out

class Armature(_Base, _generic.FuncyIncisable):
    def __init__(self, *, n = 1_000_000, **kwargs) -> None:
        self._maxn = n
        super().__init__(n = n, **kwargs)
    @property
    def value(self):
        try:
            return self._value
        except AttributeError:
            it, n = self._alliter(), self._maxn
            value = self._value = hierarchicalise_muddle(it, n)
            return value
    @property
    def shape(self):
        return self.value.shape
    def __getitem__(self, arg, /):
        return type(self)(self.value[arg])
    def __iter__(self):
        return (self[k] for k in self.value)

In [22]:
from everest.funcy import generic as _generic

myarr = np.round(np.random.rand(3, 4, 5, 6, 7), 2)

class MyArray(_generic.FuncySoftIncisable):
    def __init__(self, **kwargs):
        self.arr = myarr
        super().__init__(**kwargs)
    @property
    def shape(self):
        return self.arr.shape
    def _incision_finalise(self, args):
        return self.arr[args]

myinst = MyArray()

In [23]:
print(np.array(myinst[:2]).shape)
print(myarr[:2].shape)

(2, 4, 5, 6, 7)
(2, 4, 5, 6, 7)


In [47]:
mycut = myinst[:2, :2]

In [48]:
mycut2 = mycut[:1]

In [60]:
mycut = myinst[:2, :3]
np.array(mycut).shape

foo


(6, 5, 6, 7)

In [62]:
mycut = myinst[:2, :3][:, 1:]
np.array(mycut).shape

foo


(4, 5, 6, 7)

In [63]:
mycut = myinst[:2, :3][:, 1:][0]
np.array(mycut).shape

foo


(2, 5, 6, 7)

In [65]:
myarr[:2, :3][:, 1:][0].shape

(2, 5, 6, 7)

In [59]:
mycut = myinst[:2, :3]
np.array(mycut).shape

foo


(6, 5, 6, 7)

In [50]:
np.array(mycut).shape

foo


(4, 5, 6, 7)

In [49]:
np.array(mycut2).shape

foo


(2, 5, 6, 7)

In [44]:
np.array(mycut).shape

foo


(4, 5, 6, 7)

In [45]:
np.array(mycut[:2][:2]).shape

foo


(4, 5, 6, 7)

In [38]:
np.array(mycut).shape

foo


(4, 5, 6, 7)

In [ ]:
mycut.source

In [ ]:
list(mycut.levels[1])

In [ ]:
np.array(mycut).shape

In [ ]:
mycut.levels[0].shape

In [ ]:
len(mycut)

In [ ]:
len(list(mycut))

In [ ]:
np.array(mycut.levels[1]).shape

In [ ]:
print(np.array(myinst[:2, :2]).shape)
print(myarr[:2, :2].shape)

In [ ]:
print(np.array(myinst[:2, :2, :2]).shape)
print(myarr[:2, :2, :2].shape)

In [ ]:
print(np.array(myinst[:2, :2, :2, :2]).shape)
print(myarr[:2, :2, :2, :2].shape)

In [ ]:
print(np.array(myinst[:, :, :4][:, :, :2]).shape)
print(myarr[:, :, :4][:, :, 2].shape)

In [ ]:
np.array(myinst[:, :, :4]).shape

In [ ]:
myarr[:, :, 4].shape

In [ ]:
3 * 4

In [ ]:
testarr = np.vstack(tuple(
    myarr[inds]
        for inds in tuple(myinst[:2, :2, :2]._prime_indices()))
    )
np.allclose(testarr, np.vstack(myinst[:2, :2, :2]))

In [ ]:
print(np.array(myinst[:2, 0]).shape)
print(myarr[:2, 0].shape)

In [ ]:
print(np.array(myinst[:2, 0, :2]).shape)
print(myarr[:2, 0, :2].shape)

In [ ]:
sl = slice(1, 3, 1)
tests = [
    0,
    (0, 0),
    slice(None),
    sl,
    (sl, 0),
    (0, sl),
    (sl, 0, 0),
    (0, 0, sl),
    (0, sl, 0),
    (0, sl, 0, 0),
    (0, 0, sl, 0),
    (0, 0, sl, 0, 0),
    (0, sl, 0, sl, 0),
    (sl, sl, 0, 0, sl),
    ...,
    (..., 0),
    (0, ...),
    (..., 0, 0),
    (0, 0, ...),
    (sl, 0, ...),
    (0, sl, ...),
    (..., sl, 0),
    (..., 0, sl),
    (..., 0, sl, 0),
    (..., sl, sl, sl),
    ]
# assert all(np.allclose(np.array(myinst[test]), myarr[test]) for test in tests)
for test in tests:
    print(test)
    print(' ', np.allclose(myarr[test], np.array(myinst[test])))
#     try:
#         print(' ', np.allclose(myarr[test], np.array(myinst[test])))
#     except:
#         print('Fail')
    print('\n')

In [ ]:
# from everest.funcy.utilities import delim_split
# tests = [
#     (0,),
#     (0, 1),
#     ((0, 1), 2),
#     (0, ...),
#     (..., 0),
#     (0, 1, ...),
#     (..., 0, 1),
#     ((0, 1), ...),
#     ((0, 1), 2, ...),
#     (0, ..., 1),
#     (0, 1, ..., 2),
#     (0, ..., 1, 2),
#     (0, 1, ..., 2, 3),
#     ((0, 1), ..., 2, 3),
#     ((0, 1), ..., (2, 3)),
#     ]
# for test in tests:
#     print(test)
#     prelim, *postlim = delim_split(test)
#     postlim = postlim[0] if postlim else postlim
#     print(' ', prelim)
#     print(' ', postlim)
#     print('\n')